In [4]:
import numpy as np
import pandas as pd
from string import punctuation
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import underthesea

In [5]:
#preprocess.py
import regex as re
from underthesea import word_tokenize
import string
import codecs
from pyvi import ViTokenizer


# remove html tags
def remove_html(txt):
    return re.sub(r'<[^>]*>', '', txt)

# unicode stardard
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

# diacritic standard
vowels_to_ids = {}
vowels_table = [
    ['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a' ],
    ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
    ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
    ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e' ],
    ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
    ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i' ],
    ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o' ],
    ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
    ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
    ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u' ],
    ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
    ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y' ]
]

for i in range(len(vowels_table)):
    for j in range(len(vowels_table[i]) - 1):
        vowels_to_ids[vowels_table[i][j]] = (i, j)

def is_valid_vietnamese_word(word):
    chars = list(word)
    vowel_indexes = -1
    for index, char in enumerate(chars):
        x, y = vowels_to_ids.get(char, (-1, -1))
        if x != -1:
            if vowel_indexes == -1: vowel_indexes = index
            else:
                if index - vowel_indexes != 1: return False
                vowel_indexes = index
    return True

def standardize_word_typing(word):
    if not is_valid_vietnamese_word(word): return word
    chars = list(word)
    dau_cau = 0
    vowel_indexes = []
    qu_or_gi = False

    for index, char in enumerate(chars):
        x, y = vowels_to_ids.get(char, (-1, -1))
        if x == -1: continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True

        if y != 0:
            dau_cau = y
            chars[index] = vowels_table[x][0]

        if not qu_or_gi or index != 1:
            vowel_indexes.append(index)

    if len(vowel_indexes) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = vowels_to_ids.get(chars[1])
                chars[1] = vowels_table[x][dau_cau]
            else:
                x, y = vowels_to_ids.get(chars[2], (-1, -1))
                if x != -1: chars[2] = vowels_table[x][dau_cau]
                else: chars[1] = vowels_table[5][dau_cau] if chars[1] == 'i' else vowels_table[9][dau_cau]
            return ''.join(chars)
        return word

    for index in vowel_indexes:
        x, y = vowels_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = vowels_table[x][dau_cau]
            return ''.join(chars)

    if len(vowel_indexes) == 2:
        if vowel_indexes[-1] == len(chars) - 1:
            x, y = vowels_to_ids[chars[vowel_indexes[0]]]
            chars[vowel_indexes[0]] = vowels_table[x][dau_cau]
        else:
            x, y = vowels_to_ids[chars[vowel_indexes[1]]]
            chars[vowel_indexes[1]] = vowels_table[x][dau_cau]
    else:
        x, y = vowels_to_ids[chars[vowel_indexes[1]]]
        chars[vowel_indexes[1]] = vowels_table[x][dau_cau]
    return ''.join(chars)

def standardize_sentence_typing(text):
    words = text.lower().split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        if len(cw) == 3: cw[1] = standardize_word_typing(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

# remove unnecessary characters
def remove_unnecessary(text):
    text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬÉÈẺẼẸÊẾỀỂỄỆÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÍÌỈĨỊÚÙỦŨỤƯỨỪỬỮỰÝỲỶỸỴĐ_]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace
    return text

# special wordings in reviews (e.g., emoji,...)
def normalize_money(sent):
    return re.sub(r'[0-9]+[.,0-9][k-m-b]', 'giá', sent)

def normalize_hastag(sent):
    return re.sub(r'#+\w+', 'tag', sent)

def normalize_website(sent):
    result = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'website', sent)
    return re.sub(r'\w+(\.(com|vn|me))+((\/+([\.\w\_\-]+)?)+)?', 'website', result)

def nomalize_emoji(sent):
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', sent)

def normalize_elongate(sent):
    patern = r'(.)\1{1,}'
    result = sent
    while(re.search(patern, result) != None):
        repeat_char = re.search(patern, result)
        result = result.replace(repeat_char[0], repeat_char[1])
    return result

def remove_numbers(sent):
    return re.sub(r'[0-9]+', '', sent)

def normalize_acronyms(sent):
    text = sent
    replace_list = {
            "òa": "oà",
        "Òa": "Oà",
        "ÒA": "OÀ",
        "óa": "oá",
        "Óa": "Oá",
        "ÓA": "OÁ",
        "ỏa": "oả",
        "Ỏa": "Oả",
        "ỎA": "OẢ",
        "õa": "oã",
        "Õa": "Oã",
        "ÕA": "OÃ",
        "ọa": "oạ",
        "Ọa": "Oạ",
        "ỌA": "OẠ",
        "òe": "oè",
        "Òe": "Oè",
        "ÒE": "OÈ",
        "óe": "oé",
        "Óe": "Oé",
        "ÓE": "OÉ",
        "ỏe": "oẻ",
        "Ỏe": "Oẻ",
        "ỎE": "OẺ",
        "õe": "oẽ",
        "Õe": "Oẽ",
        "ÕE": "OẼ",
        "ọe": "oẹ",
        "Ọe": "Oẹ",
        "ỌE": "OẸ",
        "ùy": "uỳ",
        "Ùy": "Uỳ",
        "ÙY": "UỲ",
        "úy": "uý",
        "Úy": "Uý",
        "ÚY": "UÝ",
        "ủy": "uỷ",
        "Ủy": "Uỷ",
        "ỦY": "UỶ",
        "ũy": "uỹ",
        "Ũy": "Uỹ",
        "ŨY": "UỸ",
        "ụy": "uỵ",
        "Ụy": "Uỵ",
        "ỤY": "UỴ",
        'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
        'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
        'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
            #Quy các icon về 2 loại emoj: Tích cực hoặc tiêu cực
            "👹": "Tiêu cực", "👻": "Tích cực", "💃": "Tích cực",'🤙': ' Tích cực ', '👍': ' Tích cực ',
            "💄": "Tích cực", "💎": "Tích cực", "💩": "Tích cực","😕": "Tiêu cực", "😱": "Tiêu cực", "😸": "Tích cực",
            "😾": "Tiêu cực", "🚫": "Tiêu cực",  "🤬": "Tiêu cực","🧚": "Tích cực", "🧡": "Tích cực",'🐶':' Tích cực ',
            '👎': ' Tiêu cực ', '😣': ' Tiêu cực ','✨': ' Tích cực ', '❣': ' Tích cực ','☀': ' Tích cực ',
            '♥': ' Tích cực ', '🤩': ' Tích cực ', 'like': ' Tích cực ', '💌': ' Tích cực ',
            '🤣': ' Tích cực ', '🖤': ' Tích cực ', '🤤': ' Tích cực ', ':(': ' Tiêu cực ', '😢': ' Tiêu cực ',
            '❤': ' Tích cực ', '😍': ' Tích cực ', '😘': ' Tích cực ', '😪': ' Tiêu cực ', '😊': ' Tích cực ',
            '?': ' ? ', '😁': ' Tích cực ', '💖': ' Tích cực ', '😟': ' Tiêu cực ', '😭': ' Tiêu cực ',
            '💯': ' Tích cực ', '💗': ' Tích cực ', '♡': ' Tích cực ', '💜': ' Tích cực ', '🤗': ' Tích cực ',
            '^^': ' Tích cực ', '😨': ' Tiêu cực ', '☺': ' Tích cực ', '💋': ' Tích cực ', '👌': ' Tích cực ',
            '😖': ' Tiêu cực ', '😀': ' Tích cực ', ':((': ' Tiêu cực ', '😡': ' Tiêu cực ', '😠': ' Tiêu cực ',
            '😒': ' Tiêu cực ', '🙂': ' Tích cực ', '😏': ' Tiêu cực ', '😝': ' Tích cực ', '😄': ' Tích cực ',
            '😙': ' Tích cực ', '😤': ' Tiêu cực ', '😎': ' Tích cực ', '😆': ' Tích cực ', '💚': ' Tích cực ',
            '✌': ' Tích cực ', '💕': ' Tích cực ', '😞': ' Tiêu cực ', '😓': ' Tiêu cực ', '️🆗️': ' Tích cực ',
            '😉': ' Tích cực ', '😂': ' Tích cực ', ':v': '  Tích cực ', '=))': '  Tích cực ', '😋': ' Tích cực ',
            '💓': ' Tích cực ', '😐': ' Tiêu cực ', ':3': ' Tích cực ', '😫': ' Tiêu cực ', '😥': ' Tiêu cực ',
            '😃': ' Tích cực ', '😬': ' 😬 ', '😌': ' 😌 ', '💛': ' Tích cực ', '🤝': ' Tích cực ', '🎈': ' Tích cực ',
            '😗': ' Tích cực ', '🤔': ' Tiêu cực ', '😑': ' Tiêu cực ', '🔥': ' Tiêu cực ', '🙏': ' Tiêu cực ',
            '🆗': ' Tích cực ', '😻': ' Tích cực ', '💙': ' Tích cực ', '💟': ' Tích cực ',
            '😚': ' Tích cực ', '❌': ' Tiêu cực ', '👏': ' Tích cực ', ';)': ' Tích cực ', '<3': ' Tích cực ',
            '🌝': ' Tích cực ',  '🌷': ' Tích cực ', '🌸': ' Tích cực ', '🌺': ' Tích cực ',
            '🌼': ' Tích cực ', '🍓': ' Tích cực ', '🐅': ' Tích cực ', '🐾': ' Tích cực ', '👉': ' Tích cực ',
            '💐': ' Tích cực ', '💞': ' Tích cực ', '💥': ' Tích cực ', '💪': ' Tích cực ',
            '💰': ' Tích cực ',  '😇': ' Tích cực ', '😛': ' Tích cực ', '😜': ' Tích cực ',
            '🙃': ' Tích cực ', '🤑': ' Tích cực ', '🤪': ' Tích cực ','☹': ' Tiêu cực ',  '💀': ' Tiêu cực ',
            '😔': ' Tiêu cực ', '😧': ' Tiêu cực ', '😩': ' Tiêu cực ', '😰': ' Tiêu cực ', '😳': ' Tiêu cực ',
            '😵': ' Tiêu cực ', '😶': ' Tiêu cực ', '🙁': ' Tiêu cực ',
            #Chuẩn hóa 1 số sentiment words/English words
            ':))': '  Tích cực ', ':)': ' Tích cực ', 'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
            'okey': ' ok ', 'ôkê': ' ok ', 'oki': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
            ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ',
            '⭐': 'star ', '*': 'star ', '🌟': 'star ', '🎉': u' Tích cực ',
            'kg ': u' không ','not': u' không ', u' kg ': u' không ', '"k ': u' không ',' kh ':u' không ','kô':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', '"ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',
            'he he': ' Tích cực ','hehe': ' Tích cực ','hihi': ' Tích cực ', 'haha': ' Tích cực ', 'hjhj': ' Tích cực ',
            ' lol ': ' Tiêu cực ',' cc ': ' Tiêu cực ','cute': u' dễ thương ','huhu': ' Tiêu cực ', ' vs ': u' với ', 'wa': ' quá ', 'wá': u' quá', 'j': u' gì ', '“': ' ',
            ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được  ', 'dc': u' được ', 'đk': u' được ',
            'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'thick': u' Tích cực ', 'store': u' cửa hàng ',
            'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
            'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
            'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
            'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': 'tệ','fresh': ' tươi ','sad': ' tệ ',
            'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
            'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
            'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
            'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ',u'quả ng ':u' quảng  ',
            'dep': u' đẹp ',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u'qủa': u' quả ',
            'iu': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời', '><': u' Tích cực ', 'nv' : 'nhân viên', 'nvien' : 'nhân viên',
            ' por ': u' tệ ',' poor ': u' tệ ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback ', 'pùn' : 'buồn', 'tuỵt vời' : 'tuyệt vời',
            #dưới 3* quy về 1*, trên 3* quy về 5*
            '6 sao': ' 5star ','6 star': ' 5star ', '5star': ' 5star ','5 sao': ' 5star ','5sao': ' 5star ',
            'starstarstarstarstar': ' 5star ', '1 sao': ' 1star ', '1sao': ' 1star ','2 sao':' 1star ','2sao':' 1star ',
            '2 starstar':' 1star ','1star': ' 1star ', '0 sao': ' 1star ', '0star': ' 1star ',
            }

    for k, v in replace_list.items():
        text = text.replace(k, v)
    return text

# Từ điển tích cực, tiêu cực, phủ định
def load_sentiment_dicts(path_pos, path_nag, path_not):
    with codecs.open(path_pos, 'r', encoding='UTF-8') as f:
        pos = f.readlines()
    pos_list = [n.strip() for n in pos]

    with codecs.open(path_nag, 'r', encoding='UTF-8') as f:
        nag = f.readlines()
    nag_list = [n.strip() for n in nag]

    with codecs.open(path_not, 'r', encoding='UTF-8') as f:
        not_ = f.readlines()
    not_list = [n.strip() for n in not_]

    return pos_list, nag_list, not_list

# Phân tích tình cảm bằng từ điển được xác định trước
def add_sentiment_features(text, pos_list, nag_list, not_list):
    # chuyen punctuation thành space
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    text = text.translate(translator)

    text = ViTokenizer.tokenize(text)
    texts = text.split()
    len_text = len(texts)

    texts = [t.replace('_', ' ') for t in texts]
    for i in range(len_text):
        cp_text = texts[i]
        if cp_text in not_list: # Xử lý vấn đề phủ định (VD: áo này chẳng đẹp--> áo này notpos)
            numb_word = 2 if len_text - i - 1 >= 4 else len_text - i - 1

            for j in range(numb_word):
                if texts[i + j + 1] in pos_list:
                    texts[i] = 'notpos'
                    texts[i + j + 1] = ''

                if texts[i + j + 1] in nag_list:
                    texts[i] = 'notnag'
                    texts[i + j + 1] = ''
        else: #Thêm feature cho những sentiment words (áo này đẹp--> áo này đẹp Tích cực)
            if cp_text in pos_list:
                texts.append('Tích cực')
            elif cp_text in nag_list:
                texts.append('Tiêu cực')

    text = u' '.join(texts)

    #remove nốt những ký tự thừa thãi
    text = text.replace(u'"', u' ')
    text = text.replace(u'️', u'')
    text = text.replace('🏻','')
    return text

# overall preprocessing
def text_preprocess(document, pos_list, nag_list, not_list):
    #đưa về lower
    document = document.lower()
    # xóa html code
    document = remove_html(document)
    # chuẩn hóa unicode
    document = convert_unicode(document)
    
    # chuẩn hóa các ký tự đặc biệt
    document = normalize_money(document)
    document = normalize_hastag(document)
    document = normalize_website(document)
    document = nomalize_emoji(document)
    document = normalize_elongate(document)
    document = normalize_acronyms(document)
    document = remove_numbers(document)

    # chuẩn hóa cách gõ dấu tiếng việt
    document = standardize_sentence_typing(document)
    # tách từ
    document = word_tokenize(document, format="text")
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    document = remove_unnecessary(document)
    #
    document = add_sentiment_features(document, pos_list, nag_list, not_list)
    return document.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))).replace(' '*4, ' ').replace(' '*3, ' ').replace(' '*2, ' ').strip()

In [6]:
#vectorizer.py
from gensim.models import KeyedVectors

def tokenize_all_reviews(embed_model, dims, reviews):
    # split each review into a list of words
    reviews_words = [review.split() for review in reviews]

    tokenized_reviews = []
    for review in reviews_words:
        mean_embedding = np.zeros(dims)
        num_words = 0
        for word in review:
            try:
                mean_embedding += embed_model[word]
                num_words += 1
            except:
                continue
        mean_embedding /= num_words
        tokenized_reviews.append(mean_embedding)

    return tokenized_reviews 

def tokenize_reviews(embed_model, dims, sent):
    words = sent.split()
    mean_embedding = np.zeros(dims)
    num_words = 0
    for word in words:
        try:
            mean_embedding += embed_model[word]
            num_words += 1
        except:
            continue
    mean_embedding /= num_words
    return mean_embedding

class W2VLoader():
    def __init__(self, w2v_path=''):
        self.embed_model = KeyedVectors.load_word2vec_format(w2v_path, binary=False)
        self.pretrained_words = self.get_pretrained_words()
        self.vocab_size = len(self.pretrained_words)
        self.embedding_dim = self.get_embedding_dim()

    def get_pretrained_words(self):
        # Store pretrained vocab
        pretrained_words = list(self.embed_model.key_to_index.keys())
        return pretrained_words

    def get_embedding_dim(self):
        # Get dimensionality of embeddings
        word = self.pretrained_words[0]
        embedding_dim = self.embed_model.get_vector(word).shape[0]
        return embedding_dim


In [7]:
#data_loader.py
def clean_n_tokenize_data(sent, w2v):
    sent = text_preprocess(sent, pos_list, nag_list, not_list)
    tokenized_sent = tokenize_reviews(w2v.embed_model, w2v.embedding_dim, sent)
    return tokenized_sent

def read_data_from_csv():
    train = pd.read_csv('datasets/Train.csv')
    val = pd.read_csv('datasets/Val.csv')
    test = pd.read_csv('datasets/Test.csv')
    return train, val, test

def load_data(data, w2v):
    data.loc[:, 'Comments'] = data['Comments'].apply(lambda x: text_preprocess(x, pos_list, nag_list, not_list))
    tokenized_reviews = tokenize_all_reviews(w2v.embed_model, w2v.embedding_dim, data['Comments'])

    return np.stack(pd.Series(tokenized_reviews)), data[['Pin', 'Service', 'General', 'Others', 'SPin', 'SSer', 'SGeneral', 'SOth']]

In [8]:
path_pos = 'sentiment_dicts/not.txt'
path_nag = 'sentiment_dicts/nag.txt'
path_not = 'sentiment_dicts/not.txt'
pos_list, nag_list, not_list = load_sentiment_dicts(path_pos, path_nag, path_not)


In [12]:
w2v_300dims = W2VLoader(
        w2v_path='phow2v/word2vec_vi_words_300dims.txt'
    )

In [ ]:
train, val, test = read_data_from_csv()

X_train, y_train = load_data(train, w2v_300dims)
X_val, y_val = load_data(val, w2v_300dims)
X_test, y_test = load_data(test, w2v_300dims)

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

class SVCModel():

    def __init__(self, kernel='rbf', C=1.0, gamma='scale', attribute=None):
        self.kernel = kernel
        self.C = C
        self.gamma = gamma
        self.svc = None
        self.attribute = attribute

    def fit(self, X, y):
        self.svc = SVC(kernel=self.kernel, C=self.C, gamma=self.gamma, probability=True)
        self.svc.fit(X, y[self.attribute])

    def predict(self, X):
        if self.svc is None:
            raise ValueError("SVC model has not been trained yet. Please call 'fit' first.")
        return self.svc.predict(X)

    def predict_proba(self, X):
        if self.svc is None:
            raise ValueError("SVC model has not been trained yet. Please call 'fit' first.")
        return self.svc.predict_proba(X)
    
    def calculate_accuracy_score(self, X, y, y_pred=None):
        if self.svc is None:
            raise ValueError("SVC model has not been trained yet. Please call 'fit' first.")
        
        if y_pred is not None:
            accuracy = accuracy_score(y[self.attribute], y_pred)
            return round(accuracy, 2)
        
        y_pred = self.predict(X)
        accuracy = accuracy_score(y[self.attribute], y_pred)
        return round(accuracy, 2)
    
    def calculate_f1_score(self, X, y, y_pred=None):
        if self.svc is None:
            raise ValueError("SVC model has not been trained yet. Please call 'fit' first.")

        if y_pred is not None:
            if y[self.attribute].nunique() < 3:
                f1 = f1_score(y[self.attribute], y_pred)
            else:
                f1 = f1_score(y[self.attribute], y_pred, average='micro')
            return round(f1, 2)

        y_pred = self.predict(X)
        if y[self.attribute].nunique() < 3:
            f1 = f1_score(y[self.attribute], y_pred)
        else:
            f1 = f1_score(y[self.attribute], y_pred, average='micro')
        return round(f1, 2)
    
    def calculate_precision_score(self, X, y, y_pred=None):
        if self.svc is None:
            raise ValueError("SVC model has not been trained yet. Please call 'fit' first.")
        
        if y_pred is not None:
            if y[self.attribute].nunique() < 3:
                precision = precision_score(y[self.attribute], y_pred)
            else:
                precision = precision_score(y[self.attribute], y_pred, average='micro')
            return round(precision, 2)
        
        y_pred = self.predict(X)
        if y[self.attribute].nunique() < 3:
            precision = precision_score(y[self.attribute], y_pred)
        else:
            precision = precision_score(y[self.attribute], y_pred, average='micro')
        return round(precision, 2)
    
    def calculate_recall_score(self, X, y, y_pred=None):
        if self.svc is None:
            raise ValueError("SVC model has not been trained yet. Please call 'fit' first.")
        
        if y_pred is not None:
            if y[self.attribute].nunique() < 3:
                recall = recall_score(y[self.attribute], y_pred)
            else:
                recall = recall_score(y[self.attribute], y_pred, average='micro')
            return round(recall, 2)
        
        y_pred = self.predict(X)
        if y[self.attribute].nunique() < 3:
            recall = recall_score(y[self.attribute], y_pred)
        else:
            recall = recall_score(y[self.attribute], y_pred, average='micro')
        return round(recall, 2)

In [11]:
import optuna
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
import joblib
import optuna.visualization
# from model import SVCModel

class SVCTuner:
    def __init__(self, X_train, y_train, X_val, y_val, attribute=None):
        self.X_train, self.y_train, self.X_val, self.y_val = X_train, y_train, X_val, y_val
        self.attribute = attribute
        self.best_params = None
        self.best_model = None

    def objective(self, trial):
        kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
        C = trial.suggest_float('C', 1e-5, 100)
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
        
        svc_model = SVCModel(kernel=kernel, C=C, gamma=gamma, attribute=self.attribute)
        svc_model.fit(self.X_train, self.y_train)
        
        acc = svc_model.calculate_accuracy_score(self.X_val, self.y_val)
        
        return acc

    def tune(self, n_trials=100):
        study = optuna.create_study(direction='maximize')
        study.optimize(self.objective, n_trials=n_trials)
        
        self.best_params = study.best_params
        self.best_model = SVCModel(kernel=self.best_params['kernel'], 
                                   C=self.best_params['C'], 
                                   gamma=self.best_params['gamma'], 
                                   attribute=self.attribute)
        
        self.best_model.fit(self.X_train, self.y_train)
        joblib.dump(self.best_model, 'bestModel'+self.attribute)
        
        return optuna.visualization.plot_optimization_history(study)
    
    def evaluate_best_model(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test
        if self.best_model is None:
            raise ValueError("No best model available. Please run tuning first.")
        
        base_model = SVCModel(attribute=self.attribute)
        base_model.fit(self.X_train, self.y_train)
        y_pred_base = base_model.predict(self.X_test)
        base_acc = base_model.calculate_accuracy_score(self.X_test, self.y_test, y_pred=y_pred_base)
        base_f1 = base_model.calculate_f1_score(self.X_test, self.y_test, y_pred=y_pred_base)
        base_precision = base_model.calculate_precision_score(self.X_test, self.y_test, y_pred=y_pred_base)
        base_recall = base_model.calculate_recall_score(self.X_test, self.y_test, y_pred=y_pred_base)
        
        y_pred_best = self.best_model.predict(self.X_test)
        best_acc = self.best_model.calculate_accuracy_score(self.X_test, self.y_test, y_pred=y_pred_best)
        best_f1 = self.best_model.calculate_f1_score(self.X_test, self.y_test, y_pred=y_pred_best)
        best_recall = self.best_model.calculate_recall_score(self.X_test, self.y_test, y_pred=y_pred_best)
        best_precision = self.best_model.calculate_precision_score(self.X_test, self.y_test, y_pred=y_pred_best)

        return {'Base':{'Accuracy': base_acc,
                        'Precision': base_precision,
                        'Recall': base_recall,
                        'F1': base_f1},
                'Best':{'Accuracy': best_acc,
                        'Precision': best_precision,
                        'Recall': best_recall,
                        'F1': best_f1}}

In [35]:
svm_Pin = SVCTuner(X_train, y_train, X_val, y_val, attribute='Pin')
svm_Pin.tune(n_trials=100)

[I 2024-03-31 16:44:29,280] A new study created in memory with name: no-name-e995e2a6-28ae-4ffb-a7a9-5708439b0e0f
[I 2024-03-31 16:44:29,854] Trial 0 finished with value: 0.9 and parameters: {'kernel': 'linear', 'C': 82.04107601376293, 'gamma': 'scale'}. Best is trial 0 with value: 0.9.
[I 2024-03-31 16:44:30,898] Trial 1 finished with value: 0.6 and parameters: {'kernel': 'poly', 'C': 74.6052105517376, 'gamma': 'auto'}. Best is trial 0 with value: 0.9.
[I 2024-03-31 16:44:31,542] Trial 2 finished with value: 0.87 and parameters: {'kernel': 'sigmoid', 'C': 34.67100400745249, 'gamma': 'scale'}. Best is trial 0 with value: 0.9.
[I 2024-03-31 16:44:32,614] Trial 3 finished with value: 0.6 and parameters: {'kernel': 'poly', 'C': 39.33979953416238, 'gamma': 'auto'}. Best is trial 0 with value: 0.9.
[I 2024-03-31 16:44:33,175] Trial 4 finished with value: 0.9 and parameters: {'kernel': 'linear', 'C': 85.75559141890399, 'gamma': 'scale'}. Best is trial 0 with value: 0.9.
[I 2024-03-31 16:44:3

In [36]:
svm_Service = SVCTuner(X_train, y_train, X_val, y_val, attribute='Service')
svm_Service.tune(n_trials=100)

[I 2024-03-31 16:45:55,466] A new study created in memory with name: no-name-5348074d-427b-42b1-9af3-56aa4d55f476
[I 2024-03-31 16:45:56,169] Trial 0 finished with value: 0.86 and parameters: {'kernel': 'linear', 'C': 50.484544971892156, 'gamma': 'scale'}. Best is trial 0 with value: 0.86.
[I 2024-03-31 16:45:56,878] Trial 1 finished with value: 0.86 and parameters: {'kernel': 'linear', 'C': 62.64088258181133, 'gamma': 'scale'}. Best is trial 0 with value: 0.86.
[I 2024-03-31 16:45:57,945] Trial 2 finished with value: 0.72 and parameters: {'kernel': 'sigmoid', 'C': 79.55983731809141, 'gamma': 'auto'}. Best is trial 0 with value: 0.86.
[I 2024-03-31 16:45:58,646] Trial 3 finished with value: 0.87 and parameters: {'kernel': 'linear', 'C': 84.32185819628823, 'gamma': 'auto'}. Best is trial 3 with value: 0.87.
[I 2024-03-31 16:45:59,409] Trial 4 finished with value: 0.7 and parameters: {'kernel': 'sigmoid', 'C': 18.286761048614203, 'gamma': 'scale'}. Best is trial 3 with value: 0.87.
[I 20

In [37]:
svm_General = SVCTuner(X_train, y_train, X_val, y_val, attribute='General')
svm_General.tune(n_trials=100)

[I 2024-03-31 16:47:28,391] A new study created in memory with name: no-name-58a0d8b6-4e1b-43d3-a3bd-8d5b03075359
[I 2024-03-31 16:47:29,306] Trial 0 finished with value: 0.73 and parameters: {'kernel': 'linear', 'C': 93.98434787795287, 'gamma': 'auto'}. Best is trial 0 with value: 0.73.
[I 2024-03-31 16:47:30,214] Trial 1 finished with value: 0.73 and parameters: {'kernel': 'linear', 'C': 77.0804792953771, 'gamma': 'auto'}. Best is trial 0 with value: 0.73.
[I 2024-03-31 16:47:31,207] Trial 2 finished with value: 0.74 and parameters: {'kernel': 'poly', 'C': 12.749848885583976, 'gamma': 'scale'}. Best is trial 2 with value: 0.74.
[I 2024-03-31 16:47:32,298] Trial 3 finished with value: 0.66 and parameters: {'kernel': 'sigmoid', 'C': 17.699517625274336, 'gamma': 'scale'}. Best is trial 2 with value: 0.74.
[I 2024-03-31 16:47:33,694] Trial 4 finished with value: 0.72 and parameters: {'kernel': 'rbf', 'C': 75.84838661091567, 'gamma': 'scale'}. Best is trial 2 with value: 0.74.
[I 2024-03-

In [38]:
svm_Others = SVCTuner(X_train, y_train, X_val, y_val, attribute='Others')
svm_Others.tune(n_trials=100)

[I 2024-03-31 16:49:36,774] A new study created in memory with name: no-name-82c43af1-307d-46aa-88e6-dff230f93b5f
[I 2024-03-31 16:49:37,583] Trial 0 finished with value: 0.71 and parameters: {'kernel': 'poly', 'C': 5.804826284960033, 'gamma': 'auto'}. Best is trial 0 with value: 0.71.
[I 2024-03-31 16:49:38,569] Trial 1 finished with value: 0.71 and parameters: {'kernel': 'sigmoid', 'C': 42.47757137088746, 'gamma': 'auto'}. Best is trial 0 with value: 0.71.
[I 2024-03-31 16:49:39,705] Trial 2 finished with value: 0.71 and parameters: {'kernel': 'rbf', 'C': 96.7027957643594, 'gamma': 'auto'}. Best is trial 0 with value: 0.71.
[I 2024-03-31 16:49:40,837] Trial 3 finished with value: 0.71 and parameters: {'kernel': 'rbf', 'C': 85.21485140050912, 'gamma': 'auto'}. Best is trial 0 with value: 0.71.
[I 2024-03-31 16:49:41,530] Trial 4 finished with value: 0.82 and parameters: {'kernel': 'linear', 'C': 23.465582836264275, 'gamma': 'auto'}. Best is trial 4 with value: 0.82.
[I 2024-03-31 16:4

In [39]:
svm_SPin = SVCTuner(X_train, y_train, X_val, y_val, attribute='SPin')
svm_SPin.tune(n_trials=100)

[I 2024-03-31 16:50:56,706] A new study created in memory with name: no-name-5ab6aafb-2e33-4e63-a587-8532a7f62ac8
[I 2024-03-31 16:50:57,648] Trial 0 finished with value: 0.63 and parameters: {'kernel': 'sigmoid', 'C': 27.841483393907396, 'gamma': 'scale'}. Best is trial 0 with value: 0.63.
[I 2024-03-31 16:50:58,871] Trial 1 finished with value: 0.63 and parameters: {'kernel': 'sigmoid', 'C': 58.25900426835626, 'gamma': 'auto'}. Best is trial 0 with value: 0.63.
[I 2024-03-31 16:50:59,872] Trial 2 finished with value: 0.77 and parameters: {'kernel': 'poly', 'C': 37.05323152075699, 'gamma': 'scale'}. Best is trial 2 with value: 0.77.
[I 2024-03-31 16:51:01,262] Trial 3 finished with value: 0.63 and parameters: {'kernel': 'rbf', 'C': 64.09506930316269, 'gamma': 'auto'}. Best is trial 2 with value: 0.77.
[I 2024-03-31 16:51:02,475] Trial 4 finished with value: 0.63 and parameters: {'kernel': 'sigmoid', 'C': 37.366142406732564, 'gamma': 'auto'}. Best is trial 2 with value: 0.77.
[I 2024-0

In [40]:
svm_SSer = SVCTuner(X_train, y_train, X_val, y_val, attribute='SSer')
svm_SSer.tune(n_trials=100)

[I 2024-03-31 16:52:49,909] A new study created in memory with name: no-name-59c8ea0e-1597-4e2e-925b-d007235e9c16
[I 2024-03-31 16:52:50,616] Trial 0 finished with value: 0.85 and parameters: {'kernel': 'linear', 'C': 59.84878582068244, 'gamma': 'auto'}. Best is trial 0 with value: 0.85.
[I 2024-03-31 16:52:51,606] Trial 1 finished with value: 0.88 and parameters: {'kernel': 'rbf', 'C': 16.82159356444636, 'gamma': 'scale'}. Best is trial 1 with value: 0.88.
[I 2024-03-31 16:52:52,314] Trial 2 finished with value: 0.84 and parameters: {'kernel': 'linear', 'C': 43.78047796851219, 'gamma': 'scale'}. Best is trial 1 with value: 0.88.
[I 2024-03-31 16:52:53,029] Trial 3 finished with value: 0.82 and parameters: {'kernel': 'linear', 'C': 27.429397233261838, 'gamma': 'auto'}. Best is trial 1 with value: 0.88.
[I 2024-03-31 16:52:54,105] Trial 4 finished with value: 0.75 and parameters: {'kernel': 'rbf', 'C': 23.282441666224155, 'gamma': 'auto'}. Best is trial 1 with value: 0.88.
[I 2024-03-31

In [41]:
svm_SGeneral = SVCTuner(X_train, y_train, X_val, y_val, attribute='SGeneral')
svm_SGeneral.tune(n_trials=100)

[I 2024-03-31 16:54:27,377] A new study created in memory with name: no-name-569cc946-1dfc-48e1-8f0d-d76488fa1d89
[I 2024-03-31 16:54:28,471] Trial 0 finished with value: 0.61 and parameters: {'kernel': 'sigmoid', 'C': 25.102208332011084, 'gamma': 'scale'}. Best is trial 0 with value: 0.61.
[I 2024-03-31 16:54:29,465] Trial 1 finished with value: 0.68 and parameters: {'kernel': 'linear', 'C': 40.69641613608792, 'gamma': 'scale'}. Best is trial 1 with value: 0.68.
[I 2024-03-31 16:54:30,370] Trial 2 finished with value: 0.57 and parameters: {'kernel': 'poly', 'C': 53.665061221747244, 'gamma': 'auto'}. Best is trial 1 with value: 0.68.
[I 2024-03-31 16:54:31,248] Trial 3 finished with value: 0.57 and parameters: {'kernel': 'poly', 'C': 68.10801352101, 'gamma': 'auto'}. Best is trial 1 with value: 0.68.
[I 2024-03-31 16:54:32,744] Trial 4 finished with value: 0.57 and parameters: {'kernel': 'rbf', 'C': 36.29564573574274, 'gamma': 'auto'}. Best is trial 1 with value: 0.68.
[I 2024-03-31 16

In [64]:
svm_SOth = SVCTuner(X_train, y_train, X_val, y_val, attribute='SOth')
svm_SOth.tune(n_trials=100)

[I 2024-03-31 17:43:43,604] A new study created in memory with name: no-name-5e2681c9-8948-4093-909f-6584dca7db2a
[I 2024-03-31 17:43:45,471] Trial 0 finished with value: 0.48 and parameters: {'kernel': 'rbf', 'C': 94.44373348221654, 'gamma': 'auto'}. Best is trial 0 with value: 0.48.
[I 2024-03-31 17:43:47,008] Trial 1 finished with value: 0.62 and parameters: {'kernel': 'rbf', 'C': 77.90234790670965, 'gamma': 'scale'}. Best is trial 1 with value: 0.62.
[I 2024-03-31 17:43:48,170] Trial 2 finished with value: 0.62 and parameters: {'kernel': 'poly', 'C': 30.542978331607042, 'gamma': 'scale'}. Best is trial 1 with value: 0.62.
[I 2024-03-31 17:43:49,178] Trial 3 finished with value: 0.62 and parameters: {'kernel': 'linear', 'C': 36.60679378436986, 'gamma': 'auto'}. Best is trial 1 with value: 0.62.
[I 2024-03-31 17:43:50,566] Trial 4 finished with value: 0.64 and parameters: {'kernel': 'rbf', 'C': 27.242049056358695, 'gamma': 'scale'}. Best is trial 4 with value: 0.64.
[I 2024-03-31 17:

# Result

In [46]:
compare_table = {}
compare_table['Pin'] = svm_Pin.evaluate_best_model(X_test, y_test)
compare_table['Service'] = svm_Service.evaluate_best_model(X_test, y_test)
compare_table['General'] = svm_General.evaluate_best_model(X_test, y_test)
compare_table['Others'] = svm_Others.evaluate_best_model(X_test, y_test)

compare_table['SPin'] = svm_SPin.evaluate_best_model(X_test, y_test)
compare_table['SSer'] = svm_SSer.evaluate_best_model(X_test, y_test)
compare_table['SGeneral'] = svm_SGeneral.evaluate_best_model(X_test, y_test)
compare_table['SOthers'] = svm_SOth.evaluate_best_model(X_test, y_test)

In [47]:
compare_table

{'Pin': {'Base': {'Accuracy': 0.88,
   'Precision': 0.94,
   'Recall': 0.8,
   'F1': 0.86},
  'Best': {'Accuracy': 0.92, 'Precision': 0.93, 'Recall': 0.88, 'F1': 0.91}},
 'Service': {'Base': {'Accuracy': 0.83,
   'Precision': 0.96,
   'Recall': 0.38,
   'F1': 0.54},
  'Best': {'Accuracy': 0.81, 'Precision': 0.63, 'Recall': 0.72, 'F1': 0.67}},
 'General': {'Base': {'Accuracy': 0.59,
   'Precision': 0.88,
   'Recall': 0.39,
   'F1': 0.54},
  'Best': {'Accuracy': 0.68, 'Precision': 0.82, 'Recall': 0.63, 'F1': 0.71}},
 'Others': {'Base': {'Accuracy': 0.84,
   'Precision': 0.88,
   'Recall': 0.94,
   'F1': 0.91},
  'Best': {'Accuracy': 0.82, 'Precision': 0.91, 'Recall': 0.87, 'F1': 0.89}},
 'SPin': {'Base': {'Accuracy': 0.65,
   'Precision': 0.65,
   'Recall': 0.65,
   'F1': 0.65},
  'Best': {'Accuracy': 0.78, 'Precision': 0.78, 'Recall': 0.78, 'F1': 0.78}},
 'SSer': {'Base': {'Accuracy': 0.82,
   'Precision': 0.82,
   'Recall': 0.82,
   'F1': 0.82},
  'Best': {'Accuracy': 0.85, 'Precision'

In [48]:
compare_df = pd.DataFrame(compare_table).T
compare_df

,Base,Best
Pin,"{'Accuracy': 0.88, 'Precision': 0.94, 'Recall'...","{'Accuracy': 0.92, 'Precision': 0.93, 'Recall'..."
Service,"{'Accuracy': 0.83, 'Precision': 0.96, 'Recall'...","{'Accuracy': 0.81, 'Precision': 0.63, 'Recall'..."
General,"{'Accuracy': 0.59, 'Precision': 0.88, 'Recall'...","{'Accuracy': 0.68, 'Precision': 0.82, 'Recall'..."
Others,"{'Accuracy': 0.84, 'Precision': 0.88, 'Recall'...","{'Accuracy': 0.82, 'Precision': 0.91, 'Recall'..."
SPin,"{'Accuracy': 0.65, 'Precision': 0.65, 'Recall'...","{'Accuracy': 0.78, 'Precision': 0.78, 'Recall'..."
SSer,"{'Accuracy': 0.82, 'Precision': 0.82, 'Recall'...","{'Accuracy': 0.85, 'Precision': 0.85, 'Recall'..."
SGeneral,"{'Accuracy': 0.45, 'Precision': 0.45, 'Recall'...","{'Accuracy': 0.65, 'Precision': 0.65, 'Recall'..."
SOthers,"{'Accuracy': 0.61, 'Precision': 0.61, 'Recall'...","{'Accuracy': 0.68, 'Precision': 0.68, 'Recall'..."


## Several wrong predictions

In [67]:
#load model
svm_Pin = joblib.load('/kaggle/input/pretrained-svm/bestModelPin')
svm_Service = joblib.load('/kaggle/input/pretrained-svm/bestModelService')
svm_General = joblib.load('/kaggle/input/pretrained-svm/bestModelGeneral')
svm_Others = joblib.load('/kaggle/input/pretrained-svm/bestModelOthers')

svm_SPin = joblib.load('/kaggle/input/pretrained-svm/bestModelSPin')
svm_SSer = joblib.load('/kaggle/input/pretrained-svm/bestModelSSer')
svm_SGeneral = joblib.load('/kaggle/input/pretrained-svm/bestModelSGeneral')
svm_SOth = joblib.load('/kaggle/input/pretrained-svm/bestModelSOth')

In [53]:
y_pred_Pin = svm_Pin.predict(X_test)
y_pred_Service = svm_Service.predict(X_test)
y_pred_General = svm_General.predict(X_test)
y_pred_Others = svm_Others.predict(X_test)

y_pred_SPin = svm_SPin.predict(X_test)
y_pred_SSer = svm_SSer.predict(X_test)
y_pred_SGeneral = svm_SGeneral.predict(X_test)
y_pred_SOth = svm_SOth.predict(X_test)


In [54]:
pd.set_option('display.max_colwidth', None)

index_mismatch = np.where(y_pred_Pin != y_test['Pin'])[0]
Pin_mismatch = pd.DataFrame({
    'Comments': test.iloc[index_mismatch]['Comments'],
    'Predict': y_pred_Pin[index_mismatch],
    'Actual': y_test['Pin'].iloc[index_mismatch]
})
Pin_mismatch

,Comments,Predict,Actual
0,điện thoải ổn facelock cực nhanh vân tay ôk màn hình lớn pin trâu liên quân zalo youtube một ngày mất khoảng tuy chỉ chip nhưng rất mượt đa nhiệm khá ổn,1,0
8,mua được tuần và đt xài khá tốt giá tầm trung mà v là ok lắm rồi pin cực kì mạnh điểm yếu là cam trc thôi chứ cam sau đẹp lắm nha mọi người xài kĩ một tí thì máy tốt lắm chứ máy gần tr mà lên đây chê này chê nọ là sao v là tốt lắm rồi nagative,0,1
12,nâng cấp nên phiên bản mới pin sụt nhanh nghĩ mà chán nagative,1,0
14,màn hình sắc nét pin trâu chíp khỏe hình thức máy đẹp vivo s pro quá tuyệt vời,0,1
16,mình mua chưa được tuần mà tình trạng pin còn thì có đổi lại được hk với lại nhiều khi bị đứng máy thấy nản quá nagative,0,1
...,...,...,...
2169,cảm ứng quá kém cho chiếc tr không bằng chiếc samsung tr kể cả việc nhắn tin hay chơi game đều không đáp ứng được độ cảm ứng quá kém pin tạm được hiệu năng tốt chụp ảnh ok buồn nhất mỗi cảm ứng lúc mới mua được đến tuần thì ngon lắm về sau thì quá chán nagative nagative,0,1
2185,minh mua sản phẩm tại tgd lê đại hành phường đà lạt nhân viên tư vấn tốt nhiệt tình mình nhu cầu chơi game và xem youtube sản phẩm đáp ứng tốt chiến mạnh pin trâu,0,1
2187,sản phẩm tốt dung lượng pin lớn dùng được nhiều ngày loa nghe to rõ đáp ứng tốt,1,0
2198,mới chưa được tháng mà chay tgdđ trả lời giúp ae ai cùng cảnh ngộ xin cách xử lý không xạc qua đêm không vừa dùng vừa xạc nha có ai đổi được máy khác không,0,1


In [55]:
pd.set_option('display.max_colwidth', None)

index_mismatch = np.where(y_pred_Pin != y_test['Service'])[0]
Service_mismatch = pd.DataFrame({
    'Comments': test.iloc[index_mismatch]['Comments'],
    'Predict': y_pred_Service[index_mismatch],
    'Actual': y_test['Service'].iloc[index_mismatch]
})
Service_mismatch

,Comments,Predict,Actual
0,điện thoải ổn facelock cực nhanh vân tay ôk màn hình lớn pin trâu liên quân zalo youtube một ngày mất khoảng tuy chỉ chip nhưng rất mượt đa nhiệm khá ổn,0,0
1,mình mới mua vivoc tải ứng dụng games nhanh có cái không hài lòng là cài hình nền khóa màn hình không được hay tại mình chưa biết hết chức năng của nó tư vấn viên nhiệt tình,0,1
2,xấu đẹp gì không biết nhưng rất ưng tgdđ phục vụ rất tuyệt vời mua ở tgdđ mòn đít và sẽ mua dài dài ủng hộ tgdđ nagative,1,1
4,nói chung máy đẹp với màn amoled ổn trong tầm giá hơi tiếc là thời lượng pin không cao như quảng cáo,0,0
7,sản phẩm tốt pin xài cả ngày chưa hết chơi game lướt web rất mượt chụp hình đẹp cầm vừa tay rất đáng đồng tiền,0,0
...,...,...,...
2211,lần đầu nhìn máy rất ấn tượng về thiết kế camera chụp rất đẹp đúng là opo chuyên camera nên tấm nào chụp cũng đẹp pin sạc nhanh voc nên không sợ vấn đề về pin mình ghét màn hình khuyết điểm như đục lỗ giọt nước nhưng reno f đã làm tốt điều này điểm cộng là máy dùng tấm nền amoled nên màu sắc rực rỡ máy tác vụ khá mượt nhưng game thì tạm ổn nói chung mình rất hài lòng vố sản phẩm này nagative,0,0
2212,mua hôm hôm nay ngày mọi thứ ok pin cũng ngang ngửa pin mấy hãng androi tầm giaáh mà sao mấy đứa trẩu chê này chê nọ,0,0
2213,nhìn trung về chất lượng sản phẩm còn thua xa opo của trung quốc đung thì hay bị đơ thôi chót đại giờ thì cứ samsung mà dùng khỏi phải lo nagative nagative,0,1
2214,giá hợp lý cấu hình tốt nhìn chung hài lòng về sản phẩm mong vinsmart tiếp tục ra các sản phẩm cao cấp hơn nữa,1,1


In [56]:
index_mismatch = np.where(y_pred_General != y_test['General'])[0]
General_mismatch = pd.DataFrame({
    'Comments': test.iloc[index_mismatch]['Comments'],
    'Predict': y_pred_General[index_mismatch],
    'Actual': y_test['General'].iloc[index_mismatch]
})
General_mismatch

,Comments,Predict,Actual
0,điện thoải ổn facelock cực nhanh vân tay ôk màn hình lớn pin trâu liên quân zalo youtube một ngày mất khoảng tuy chỉ chip nhưng rất mượt đa nhiệm khá ổn,0,1
2,xấu đẹp gì không biết nhưng rất ưng tgdđ phục vụ rất tuyệt vời mua ở tgdđ mòn đít và sẽ mua dài dài ủng hộ tgdđ nagative,1,0
4,nói chung máy đẹp với màn amoled ổn trong tầm giá hơi tiếc là thời lượng pin không cao như quảng cáo,1,0
5,đt có lỗi là chế dô bỏ túi là ánh sáng điện thoại là nhấp đúp vào màn hình thì không nhạy cho lắm còn lại thì rất tốt nagative,0,1
14,màn hình sắc nét pin trâu chíp khỏe hình thức máy đẹp vivo s pro quá tuyệt vời,0,1
...,...,...,...
2203,máy dùng ok chê mỗi cái loa ngoài âm thanh hơi chán mong nsx lưu ý khắc phục ở những phiên bản máy sau nagative,0,1
2206,máy đã sử dụng mà pin rất trâu tác vụ nhanh mượt không có gì để chê,1,0
2211,lần đầu nhìn máy rất ấn tượng về thiết kế camera chụp rất đẹp đúng là opo chuyên camera nên tấm nào chụp cũng đẹp pin sạc nhanh voc nên không sợ vấn đề về pin mình ghét màn hình khuyết điểm như đục lỗ giọt nước nhưng reno f đã làm tốt điều này điểm cộng là máy dùng tấm nền amoled nên màu sắc rực rỡ máy tác vụ khá mượt nhưng game thì tạm ổn nói chung mình rất hài lòng vố sản phẩm này nagative,0,1
2213,nhìn trung về chất lượng sản phẩm còn thua xa opo của trung quốc đung thì hay bị đơ thôi chót đại giờ thì cứ samsung mà dùng khỏi phải lo nagative nagative,0,1


In [57]:
index_mismatch = np.where(y_pred_Others != y_test['Others'])[0]
Others_mismatch = pd.DataFrame({
    'Comments': test.iloc[index_mismatch]['Comments'],
    'Predict': y_pred_Others[index_mismatch],
    'Actual': y_test['Others'].iloc[index_mismatch]
})
Others_mismatch

,Comments,Predict,Actual
9,pin nhanh hết vân tay cũng chưa được nhạy nhân vien tgd tư vấn nhiệt tình dễ thương star về phong cách bán hàng,1,0
13,đang dùng tháng ok sang tháng thư bắt đau lỗi chán quá săp nguồn chết không lên được lại còn cứ kêu tắt tình ắng chế tài nghe gì ý nagative nagative nagative,0,1
15,dùng hơn tuần cảm giác mọi thứ ok chỉ có sóng sánh hơi kém hay mất sóng có cách nào khắc phục không qtv mong phản hồi,0,1
22,máy cũng xài tạm ổn màn hình nó không được nhay nhiều lúc phải bấm vài lần mới được,1,0
28,sau vài ngày sử dụng sản phẩm thấy đt cx tạm ổn nếu vừa xài vừa xạc thì cảm ứq sẽ bị đơ pin tuột nhanh vân tay xài cx khá ổn nagative nagative,0,1
...,...,...,...
2204,em không hiểu sao điện thoại dùng chưa được tháng mà bữa nay tự tắt nguồn vậy ạ,0,1
2205,máy mới mua về xài rất ổn định nhân viên rất nhiệt tình khách hàng thân thiết được giảm thêm giá máy nữa,1,0
2206,máy đã sử dụng mà pin rất trâu tác vụ nhanh mượt không có gì để chê,0,1
2212,mua hôm hôm nay ngày mọi thứ ok pin cũng ngang ngửa pin mấy hãng androi tầm giaáh mà sao mấy đứa trẩu chê này chê nọ,0,1


In [58]:
index_mismatch = np.where(y_pred_SPin != y_test['SPin'])[0]
SPin_mismatch = pd.DataFrame({
    'Comments': test.iloc[index_mismatch]['Comments'],
    'Predict': y_pred_SPin[index_mismatch],
    'Actual': y_test['SPin'].iloc[index_mismatch]
})
SPin_mismatch

,Comments,Predict,Actual
0,điện thoải ổn facelock cực nhanh vân tay ôk màn hình lớn pin trâu liên quân zalo youtube một ngày mất khoảng tuy chỉ chip nhưng rất mượt đa nhiệm khá ổn,1.0,0
4,nói chung máy đẹp với màn amoled ổn trong tầm giá hơi tiếc là thời lượng pin không cao như quảng cáo,1.0,-1
8,mua được tuần và đt xài khá tốt giá tầm trung mà v là ok lắm rồi pin cực kì mạnh điểm yếu là cam trc thôi chứ cam sau đẹp lắm nha mọi người xài kĩ một tí thì máy tốt lắm chứ máy gần tr mà lên đây chê này chê nọ là sao v là tốt lắm rồi nagative,0.0,1
9,pin nhanh hết vân tay cũng chưa được nhạy nhân vien tgd tư vấn nhiệt tình dễ thương star về phong cách bán hàng,1.0,-1
12,nâng cấp nên phiên bản mới pin sụt nhanh nghĩ mà chán nagative,-1.0,0
...,...,...,...
2201,thua về mọi mặc trong tầm giá mua được tháng đơ lag cam đã yếu chơi game yếu chỉ được cái pin ok nagative nagative nagative,0.0,1
2203,máy dùng ok chê mỗi cái loa ngoài âm thanh hơi chán mong nsx lưu ý khắc phục ở những phiên bản máy sau nagative,-1.0,0
2209,theo trải nghiệm của tôi thì tầm giá này thì mọi thứ khá là ok nhưng chỉ có chút vấn đề là sạc pin rất là nóng tôi vừa mới mua được vài giờ đồng hồ,1.0,-1
2212,mua hôm hôm nay ngày mọi thứ ok pin cũng ngang ngửa pin mấy hãng androi tầm giaáh mà sao mấy đứa trẩu chê này chê nọ,-1.0,1


In [59]:
index_mismatch = np.where(y_pred_SSer != y_test['SSer'])[0]
SSer_mismatch = pd.DataFrame({
    'Comments': test.iloc[index_mismatch]['Comments'],
    'Predict': y_pred_SSer[index_mismatch],
    'Actual': y_test['SSer'].iloc[index_mismatch]
})
SSer_mismatch

,Comments,Predict,Actual
1,mình mới mua vivoc tải ứng dụng games nhanh có cái không hài lòng là cài hình nền khóa màn hình không được hay tại mình chưa biết hết chức năng của nó tư vấn viên nhiệt tình,0.0,1
6,thật tuyệt máy qua mượt túi thích như thế giới di động có chỗ giá khác túi không hiểu đánh giá star cho thế giới di động phương phú châu star nhân viên ok,1.0,0
10,mới xài để cặp đã nứt màn không lên nứt nên không có bảo hành thay màn gần bằng máy mới chán nagative,0.0,-1
25,máy quá kém chơi game thì giật giật máy mau nóng quá không hợp với giá triệu đó giờ xài opo không giờ mua samsung không dám mua cái thứ quá tệ nagative,0.0,-1
26,khá ngon trong tầm giá màn hình lớn chơi game xem phim ngon chip khá mạnh chỉ hơi ấm tí,1.0,0
...,...,...,...
2161,sản phẩm rất ok mượt thiết kế đẹp mắt chất lưong sản phẩm rất ok hàng chính hãng giá cả hợp lý cho một sản phẩm như vâyh,-1.0,0
2164,sang tuần dự định mua vậy sau giảm giá rồi mình phải chi tổng bao nhiêu tiền đang phân vân về giá,-1.0,0
2177,máy xài được tuần pin thì sai trung bình xiaomi pin về sao mới lâu cấu hình chơi hero survival người combo không lag là ok camera như cậy thì được rồi đối với mình mình thấy giá máy như vậy là ok lun nhân viên ở bạc liêu thì nhiệt tình và hết mình hổ trợ cho khách mình cảm thấy vui khi mua đồ ở thế giới di động nagative nagative,0.0,1
2190,mới mua được tuần về cứ bị treo chữ samsung là sao ạ không lẽ bán lỗ thất vọng quá nagative nagative,-1.0,0


In [60]:
index_mismatch = np.where(y_pred_SGeneral != y_test['SGeneral'])[0]
SGeneral_mismatch = pd.DataFrame({
    'Comments': test.iloc[index_mismatch]['Comments'],
    'Predict': y_pred_SGeneral[index_mismatch],
    'Actual': y_test['SGeneral'].iloc[index_mismatch]
})
SGeneral_mismatch

,Comments,Predict,Actual
2,xấu đẹp gì không biết nhưng rất ưng tgdđ phục vụ rất tuyệt vời mua ở tgdđ mòn đít và sẽ mua dài dài ủng hộ tgdđ nagative,1,0
4,nói chung máy đẹp với màn amoled ổn trong tầm giá hơi tiếc là thời lượng pin không cao như quảng cáo,1,0
5,đt có lỗi là chế dô bỏ túi là ánh sáng điện thoại là nhấp đúp vào màn hình thì không nhạy cho lắm còn lại thì rất tốt nagative,0,1
12,nâng cấp nên phiên bản mới pin sụt nhanh nghĩ mà chán nagative,0,-1
13,đang dùng tháng ok sang tháng thư bắt đau lỗi chán quá săp nguồn chết không lên được lại còn cứ kêu tắt tình ắng chế tài nghe gì ý nagative nagative nagative,0,-1
...,...,...,...
2206,máy đã sử dụng mà pin rất trâu tác vụ nhanh mượt không có gì để chê,1,0
2211,lần đầu nhìn máy rất ấn tượng về thiết kế camera chụp rất đẹp đúng là opo chuyên camera nên tấm nào chụp cũng đẹp pin sạc nhanh voc nên không sợ vấn đề về pin mình ghét màn hình khuyết điểm như đục lỗ giọt nước nhưng reno f đã làm tốt điều này điểm cộng là máy dùng tấm nền amoled nên màu sắc rực rỡ máy tác vụ khá mượt nhưng game thì tạm ổn nói chung mình rất hài lòng vố sản phẩm này nagative,0,1
2213,nhìn trung về chất lượng sản phẩm còn thua xa opo của trung quốc đung thì hay bị đơ thôi chót đại giờ thì cứ samsung mà dùng khỏi phải lo nagative nagative,0,-1
2221,sản phẩm tốt ai chơi game cứ mang em về mà chơi nhé ngắn gọn thế thôu,0,1


In [61]:
index_mismatch = np.where(y_pred_SOth != y_test['SOth'])[0]
SOth_mismatch = pd.DataFrame({
    'Comments': test.iloc[index_mismatch]['Comments'],
    'Predict': y_pred_SOth[index_mismatch],
    'Actual': y_test['SOth'].iloc[index_mismatch]
})
SOth_mismatch

,Comments,Predict,Actual
2,xấu đẹp gì không biết nhưng rất ưng tgdđ phục vụ rất tuyệt vời mua ở tgdđ mòn đít và sẽ mua dài dài ủng hộ tgdđ nagative,1.0,0
5,đt có lỗi là chế dô bỏ túi là ánh sáng điện thoại là nhấp đúp vào màn hình thì không nhạy cho lắm còn lại thì rất tốt nagative,1.0,-1
6,thật tuyệt máy qua mượt túi thích như thế giới di động có chỗ giá khác túi không hiểu đánh giá star cho thế giới di động phương phú châu star nhân viên ok,1.0,0
8,mua được tuần và đt xài khá tốt giá tầm trung mà v là ok lắm rồi pin cực kì mạnh điểm yếu là cam trc thôi chứ cam sau đẹp lắm nha mọi người xài kĩ một tí thì máy tốt lắm chứ máy gần tr mà lên đây chê này chê nọ là sao v là tốt lắm rồi nagative,1.0,0
11,máy thiết kế quá đẹp dùng được mấy hôm rồi thấy máy vẫn ổn tiếc là pin tụt quá nhanh pin sạc thì nhanh nóng,0.0,1
...,...,...,...
2203,máy dùng ok chê mỗi cái loa ngoài âm thanh hơi chán mong nsx lưu ý khắc phục ở những phiên bản máy sau nagative,-1.0,0
2204,em không hiểu sao điện thoại dùng chưa được tháng mà bữa nay tự tắt nguồn vậy ạ,0.0,-1
2205,máy mới mua về xài rất ổn định nhân viên rất nhiệt tình khách hàng thân thiết được giảm thêm giá máy nữa,1.0,0
2209,theo trải nghiệm của tôi thì tầm giá này thì mọi thứ khá là ok nhưng chỉ có chút vấn đề là sạc pin rất là nóng tôi vừa mới mua được vài giờ đồng hồ,1.0,0


# Demo

In [10]:
# from data_loader import clean_n_tokenize_data
# from vectorizer import W2VLoader
import gradio as gr
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import joblib

#load model
svm_Pin = joblib.load('/kaggle/input/pretrained-svm/bestModelPin')
svm_Service = joblib.load('/kaggle/input/pretrained-svm/bestModelService')
svm_General = joblib.load('/kaggle/input/pretrained-svm/bestModelGeneral')
svm_Others = joblib.load('/kaggle/input/pretrained-svm/bestModelOthers')

svm_SPin = joblib.load('/kaggle/input/pretrained-svm/bestModelSPin')
svm_SSer = joblib.load('/kaggle/input/pretrained-svm/bestModelSSer')
svm_SGeneral = joblib.load('/kaggle/input/pretrained-svm/bestModelSGeneral')
svm_SOth = joblib.load('/kaggle/input/pretrained-svm/bestModelSOth')

# w2v_300dims = W2VLoader(
#     w2v_path='/kaggle/input/phow2v/word2vec_vi_words_300dims/word2vec_vi_words_300dims.txt'
# )

#demo
aspects = ['Pin', 'Service', 'General', 'Others']
aspects_ratio = []

sentiments = ['Tích cực', 'Tiêu cực', 'Neutral']
sentiments_ratio = []

def sentiment_analysis(sent):
    token_sent = clean_n_tokenize_data(sent, w2v_300dims).reshape(1, -1)

    pin_prob = svm_Pin.predict_proba(token_sent).ravel()
    service_prob = svm_Service.predict_proba(token_sent).ravel()
    general_prob = svm_General.predict_proba(token_sent).ravel()
    others_prob = svm_Others.predict_proba(token_sent).ravel()

    spin_prob = svm_SPin.predict_proba(token_sent).ravel()
    sser_prob = svm_SSer.predict_proba(token_sent).ravel()
    sgen_prob = svm_SGeneral.predict_proba(token_sent).ravel()
    soth_prob = svm_SOth.predict_proba(token_sent).ravel()

    aspects_ratio.clear()
    sentiments_ratio.clear()
    aspects_ratio.extend([pin_prob, service_prob, general_prob, others_prob])
    sentiments_ratio.extend([spin_prob, sser_prob, sgen_prob, soth_prob])

def draw_bar_aspect():
    # Tạo list chứa các thanh bar
    bars = []

    # Vẽ từng thanh bar cho mỗi khía cạnh
    for i in range(len(aspects)):
        bars.append(go.Bar(
            y=[aspects[i]],
            x=[aspects_ratio[i][0]],
            orientation='h',
            name='Không',
            showlegend=True if i == 0 else False,
            marker=dict(color='orange')
        ))
        bars.append(go.Bar(
            y=[aspects[i]],
            x=[aspects_ratio[i][1]],
            orientation='h',
            name='Có',
            showlegend=True if i == 0 else False,
            marker=dict(color='blue')
        ))

    # Tạo layout
    layout = go.Layout(
        title='Aspects',
        barmode='stack',
        xaxis=dict(title='Ratio'),
        yaxis=dict(title='Aspects')
    )

    # Tạo figure
    fig = go.Figure(data=bars, layout=layout)

    return fig

def draw_bar_sentiment():
    bars = []

    # Vẽ từng thanh bar cảm xúc cho mỗi khía cạnh
    for i in range(len(aspects)):
        bars.append(go.Bar(
            y=[aspects[i]],
            x=[sentiments_ratio[i][0]],
            orientation='h',
            name='Tiêu cực',
            showlegend=True if i == 0 else False,
            marker=dict(color='orange')
        ))
        bars.append(go.Bar(
            y=[aspects[i]],
            x=[sentiments_ratio[i][1]],
            orientation='h',
            name='Neutral',
            showlegend=True if i == 0 else False,
            marker=dict(color='blue')
        ))
        bars.append(go.Bar(
            y=[aspects[i]],
            x=[sentiments_ratio[i][2]],
            orientation='h',
            name='Tích cực',
            showlegend=True if i == 0 else False,
            marker=dict(color='red')
        ))

    # Tạo layout
    layout = go.Layout(
        title='Sentiments of Aspects',
        barmode='stack',
        xaxis=dict(title='Ratio'),
        yaxis=dict(title='Aspects')
    )

    # Tạo figure
    fig = go.Figure(data=bars, layout=layout)

    return fig

def draw_pie_aspect():
    # Tạo subplot với loại 'pie' và khoảng trống
    fig = make_subplots(rows=1, cols=4, subplot_titles=aspects, horizontal_spacing=0.05, specs=[[{'type':'pie'}]*4])

    # Vẽ biểu đồ pie cho mỗi khía cạnh
    for i, aspect in enumerate(aspects):
        labels = ['Không', 'Có']
        values = aspects_ratio[i]
        
        # Tạo biểu đồ pie và thêm vào subplot tương ứng
        fig.add_trace(go.Pie(labels=labels, values=values, name=aspect), row=1, col=i+1)

    # Cài đặt layout
    fig.update_layout(
        title='Aspect',
    )
    return fig

def draw_pie_sentiment():
    # Tạo subplot với loại 'pie' và khoảng trống
    fig = make_subplots(rows=1, cols=4, subplot_titles=aspects, horizontal_spacing=0.05, specs=[[{'type':'pie'}]*4])

    # Vẽ biểu đồ pie cho mỗi khía cạnh
    for i, aspect in enumerate(aspects):
        labels = sentiments
        values = sentiments_ratio[i]
        
        # Tạo biểu đồ pie và thêm vào subplot tương ứng
        fig.add_trace(go.Pie(labels=labels, values=values, name=aspect), row=1, col=i+1)

    # Cài đặt layout
    fig.update_layout(
        title='Sentiment Analysis by Aspect',
    )
    return fig

def draw_plot_aspect(type_plot):
    if type_plot == 'Bar':
        return draw_bar_aspect()
    else:
        return draw_pie_aspect()
    
def draw_plot_sentiment(type_plot):
    if type_plot == 'Bar':
        return draw_bar_sentiment()
    else:
        return draw_pie_sentiment()

def draw_plot(type_aspect_plot, type_sentiment_plot):
    return draw_plot_aspect(type_aspect_plot), draw_plot_sentiment(type_sentiment_plot)

def submit(comment, type_aspect_plot, type_sentiment_plot):
    sentiment_analysis(comment)
    plot = draw_plot(type_aspect_plot, type_sentiment_plot)
    return {
        output_plot1: gr.Column(visible=True),
        plot_aspect: plot[0],
        plot_sentiment: plot[1],
    }
    

with gr.Blocks() as demo:
    with gr.Row():
        text_box = gr.Textbox(placeholder="Write your comment here...", visible=True, label="Comment")
        submit_btn = gr.Button(value="Submit")
    with gr.Row(visible=False) as output_plot1:
        with gr.Column():
            choose_plot_aspect_dropdown = gr.Dropdown(
                    choices=['Bar', 'Pie'], 
                    label="Choose Plot Type",
                    value='Bar',
                    )
            plot_aspect = gr.Plot()
        with gr.Column():
            choose_plot_sentiment_dropdown = gr.Dropdown(
                    choices=['Bar', 'Pie'], 
                    label="Choose Plot Type",
                    value='Pie',
                    )
            plot_sentiment = gr.Plot() 

    choose_plot_aspect_dropdown.select(
            fn = draw_plot_aspect,
            inputs = [choose_plot_aspect_dropdown],
            outputs = [plot_aspect],
        )
    choose_plot_sentiment_dropdown.select(
            fn = draw_plot_sentiment,
            inputs = [choose_plot_sentiment_dropdown],
            outputs = [plot_sentiment],
        )

    submit_btn.click(
    fn = submit,
    inputs = [text_box, choose_plot_aspect_dropdown, choose_plot_sentiment_dropdown],
    outputs = [output_plot1, plot_aspect, plot_sentiment],
)

demo.launch(share=False)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
